# NLLB use for translation task 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline,DataCollatorForSeq2Seq,EarlyStoppingCallback, TrainingArguments,AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments,TrainerCallback, Trainer, DataCollatorForLanguageModeling, logging
import pandas as pd
import json
from sklearn.metrics import cohen_kappa_score
import torch
from tqdm.notebook import tqdm
import os
from huggingface_hub import login
from datasets import load_dataset

In [ ]:
dataset = load_dataset("csv", data_files="/kaggle/input/dataset-concatenato/dataset_concatenato.csv")["train"]
dataset = dataset.train_test_split(test_size=0.3)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
path_NNLB="/kaggle/working/nllb-finetuned-antico-moderno"
os.makedirs(path_NNLB, exist_ok=True)

# Import model and setup language

In [ ]:
model_name = "facebook/nllb-200-distilled-600M"

src_lang = tgt_lang = "ita_Latn"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.src_lang = "ita_Latn"
forced_bos_token_id = tokenizer.convert_tokens_to_ids("ita_Latn")

# Tokenize text

In [ ]:
max_len=256
def preprocess(example):
    model_inputs = tokenizer(
        example["text"],
        max_length=max_len,
        truncation=True,
        padding="max_length"
    )
    labels = tokenizer(
        example["translation"],
        max_length=max_len,
        truncation=True,
        padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
tokenized_dataset = {
    "train": dataset["train"].map(preprocess, remove_columns=dataset["train"].column_names),
    "test": dataset["test"].map(preprocess, remove_columns=dataset["test"].column_names)
}

# Realize trainer 

In [ ]:
class ClearCUDACacheCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/working/nllb-finetuned-antico-moderno",
    eval_strategy="epoch",
    save_strategy="epoch",
    gradient_accumulation_steps=4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=2e-4,
    num_train_epochs=8,
    predict_with_generate=True,
    load_best_model_at_end=True,
    fp16=True, 
    logging_dir="/kaggle/working/nllb-finetuned-antico-moderno/logs",
    save_total_limit=1,
    seed=42,
    report_to="none",
    metric_for_best_model="eval_loss",
    generation_max_length=max_len,
    generation_num_beams=4
)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
def compute_metrics(eval_preds):
    return {} 
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[ClearCUDACacheCallback(),EarlyStoppingCallback(early_stopping_patience=1)]
)

# Training Steps

In [ ]:
trainer.train()
metrics = trainer.evaluate()
print(metrics)
model.save_pretrained("/kaggle/working/nllb-finetuned-antico-moderno")
tokenizer.save_pretrained("/kaggle/working/nllb-finetuned-antico-moderno")

# inference steps 

In [ ]:
df = pd.read_csv("/kaggle/input/dataset-c/dataset_cleaned.csv")
df.head()

In [ ]:
model_path="/kaggle/working/nllb-finetuned-antico-moderno"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path,
    device_map="auto")
tokenizer.pad_token = tokenizer.eos_token
model.eval().to(device)


df["generated_translation"] = ""

# 3. Define translation function
def traduci_seq_to_seq(s):
    src_lang = "ita_Latn"
    tgt_lang = "ita_Latn"

    # imposta la lingua sorgente
    tokenizer.src_lang = src_lang
    inputs = tokenizer(s, return_tensors="pt", truncation=True,max_length=384).to(model.device)
    forced_bos_token_id = tokenizer.convert_tokens_to_ids(tgt_lang)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            use_cache=True,
            num_beams=4,
            forced_bos_token_id=forced_bos_token_id
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# 4. Process with progress bar
results = []
i =0
for s in tqdm(df["Sentence"].tolist()):
    results.append(traduci_seq_to_seq(s))
        
df["generated_translation"] = results

path_translations="/kaggle/working/translations"
os.makedirs(path_translations, exist_ok=True)
# 5. Save result
translation_file=path_translations+"dataset_with_translation_NNLB.csv"
df.to_csv(translation_file, index=False)

In [ ]:
df.head()